In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def Naver_news_search_crawl(keyword, date_start, date_end, sub=True):
  main_url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=" + keyword + "&nso=so%3Ar%2Cp%3Afrom" + date_start +"to" + date_end
  main_page = BeautifulSoup(requests.get(main_url).text, 'lxml')

  news = []
  while True:
    article = main_page.find_all('li')

    for a in article:
      if a.find('a', {'class':'news_tit'}):
        news.append(a.find_all('a', {'class':'news_tit'}))
        if sub==True:
          if a.find('a', {'class':'elss sub_tit'}):
            news.append(a.find_all('a', {'class':'elss sub_tit'}))

    if main_page.find('a', {'class':'btn_next'})['aria-disabled']=='true':
      break

    main_url = "https://search.naver.com/search.naver?" + main_page.find('a', {'class':'btn_next'})['href']
    main_page = BeautifulSoup(requests.get(main_url).text, 'lxml')
  
  news = [n for N in news for n in N]

  news_url = [n['href'] for n in news]

  news_title = [n.text for n in news]

  # save
  data = pd.DataFrame({
          "title": news_title,
          "url": news_url
          })
  
  if sub==True:
    data.to_csv(keyword + '_' + date_start + '_' + date_end + '.csv',index=True)
  else:
    data.to_csv(keyword + '_' + date_start + '_' + date_end + '_main.csv',index=True)

In [2]:
def Naver_RankingNews_crawl(date):
  # main page
  main_url = "https://news.naver.com/main/ranking/popularDay.nhn?date=" + date
  main_page = BeautifulSoup(requests.get(main_url, headers={'User-Agent':'Mozilla/5.0'}).text, 'html.parser')

  news = main_page.find_all('div', {'class':'list_content'})

  news_url = ['https://news.naver.com' + n.find('a')['href'] for n in news]
  news_title = [n.find('a').text for n in news]
  
  news_content = []
  for url in news_url:
    # page
    article_url = url
    article_page = BeautifulSoup(requests.get(article_url, headers={'User-Agent':'Mozilla/5.0'}).text, 'html.parser')

    # content
    article = article_page.find('div', {'id':'articleBodyContents'}).text.replace('\n', ' ')
    content = article.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_content.append(content)

    # comment
    comment_url = article_url.replace('ranking/read.nhn?', 'read.nhn?m_view=1&includeAllCount=true&')
    comment_page = BeautifulSoup(requests.get(comment_url, headers={'User-Agent':'Mozilla/5.0'}).text, 'html.parser')

  # save
  data = pd.DataFrame({
          "title": news_title,
          "content": news_content
          })
  
  data.to_csv(date + '_Naver_RankingNews.csv',index=True)

Naver_RankingNews_crawl('20210130')